In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
BATCH_SIZE = 32  # Number of training examples to process before updating our models variables
IMG_SHAPE  = 150  # Our training data consists of images with width of 150 pixels and height of 150 pixels
total_train = 10343
total_val = 4424

#DESIRED_ACCURACY = 0.98

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        #if(logs.get('accuracy') > DESIRED_ACCURACY):
        if logs.get('acc') is not None and logs.get('acc') > 0.98:
            print("\nReached 98% accuracy so cencelling training!")
            self.model.stop_training = True

callbacks = myCallback()

In [ ]:
train_image_generator = ImageDataGenerator(
      rescale = 1./255,
	  rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_image_generator = ImageDataGenerator(
      rescale = 1./255,
	  rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                           directory='/content/drive/My Drive/Deep Learning/Natural Face/train',
                                                           target_size=(IMG_SHAPE,IMG_SHAPE), 
                                                           shuffle=True,
                                                           class_mode='binary')

Found 10343 images belonging to 7 classes.


In [ ]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                              directory='/content/drive/My Drive/Deep Learning/Natural Face/test',
                                                              shuffle=False,
                                                              target_size=(IMG_SHAPE,IMG_SHAPE), 
                                                              class_mode='binary')

Found 4424 images belonging to 7 classes.


In [ ]:
callbacks = myCallback()
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPool2D(2,2),
    
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(7, activation='sigmoid')
])
model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 18496)            

In [ ]:
from keras import backend as K

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy',precision_m])

In [ ]:
epochs = 5
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(train_data_gen.n / float(BATCH_SIZE))),
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(val_data_gen.n / float(BATCH_SIZE)))
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
324/324 [==============================] - 3149s 10s/step - loss: 1.8951 - accuracy: 0.2497 - precision_m: 1.5774 - val_loss: 1.8167 - val_accuracy: 0.2801 - val_precision_m: 1.6276
Epoch 2/5
324/324 [==============================] - 310s 955ms/step - loss: 1.8226 - accuracy: 0.2824 - precision_m: 1.7476 - val_loss: 1.7791 - val_accuracy: 0.3092 - val_precision_m: 1.5669
Epoch 3/5
324/324 [==============================] - 308s 949ms/step - loss: 1.7473 - accuracy: 0.3146 - precision_m: 1.6499 - val_loss: 1.7016 - val_accuracy: 0.3402 - val_precision_m: 1.5752
Epoch 4/5
324/324 [==============================] - 310s 957ms/step - loss: 1.6770 - accuracy: 0.3541 - precision_m: 1.6267 - val_loss: 1.6310 - val_accuracy: 0.3655 - val_precision_m: 1.6212
Epoch 5/5
324/324 [==============================] - 310s 957ms/step - loss: 1.6274 - accuracy: 0.3689 - precision_m: 1.6148 - val_loss: 1.6195 - val_accuracy: 0.3705 - val_precision_m: 1.6194
